In [1]:
import os
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
import subreddit_finetuning as sub_fin

# Generate Questions for Benchmark

In [ ]:
sub_fin.generate_for_benchmark()

# Generate Questions for Finetuning

In [ ]:
sub_fin.generate_for_finetuning()